In [1]:
##############################################
###  Train & Register SAS Logit IML Model  ###
##############################################

###################
### Credentials ###
###################

import os
import sys
from pathlib import Path

sys.path.append('C:/Users/chparr/OneDrive - SAS/credentials')
from credentials import hostname, session, port, protocol, wd, output_dir, git_dir, token_dir, token, token_refresh, token_pem, username

In [2]:
#############################
### Connect with SAS Viya ###
#############################

import swat

access_token = open(token, "r").read()
conn =  swat.CAS(hostname=hostname, username=None, password=access_token, ssl_ca_list=token_pem, protocol=protocol)
print(conn.serverstatus())

NOTE: Grid node action status report: 1 nodes, 9 total actions executed.
[About]

 {'CAS': 'Cloud Analytic Services',
  'CASCacheLocation': 'CAS Disk Cache',
  'CASHostAccountRequired': 'OPTIONAL',
  'Copyright': 'Copyright © 2014-2024 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2024-05-02T18:52:31Z',
  'System': {'Hostname': 'controller.sas-cas-server-default.innovationlab.svc.cluster.local',
   'Linux Distribution': 'Red Hat Enterprise Linux release 8.9 (Ootpa)',
   'Model Number': 'x86_64',
   'OS Family': 'LIN X64',
   'OS Name': 'Linux',
   'OS Release': '5.15.0-1042-azure',
   'OS Version': '#49-Ubuntu SMP Tue Jul 11 17:28:46 UTC 2023'},
  'Transferred': 'NO',
  'Version': '4.00',
  'VersionLong': 'V.04.00M0P04152024',
  'Viya Release': '20240420.1713587951902',
  'Viya Version': 'Stable 2024.04',
  'license': {'expires': '06Sep2024:00:00:00',
   'gracePeriod': 0,
   'site': 'CIS SSEMONTHLY INNOVATION ENTERPRISE-RISK-MRM',
   'siteNum': 70180938,
   'warningPeriod'

In [3]:
#############################
### Identify Table in CAS ###
#############################

### caslib and table to use in modeling
caslib = 'casuser'
in_mem_tbl = 'AML_BANK_PREP'

### load table in-memory if not already exists in-memory
if conn.table.tableExists(caslib=caslib, name=in_mem_tbl).exists<=0:
    conn.table.loadTable(caslib=caslib, path=str(in_mem_tbl+str('.sashdat')), 
                         casout={'name':in_mem_tbl, 'caslib':caslib, 'promote':True})

### show table to verify
conn.table.tableInfo(caslib=caslib, wildIgnore=False, name=in_mem_tbl)

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime,TableRedistUpPolicy
0,AML_BANK_PREP,14302,27,0,utf-8,2024-04-30T14:36:02+00:00,2024-04-30T14:36:02+00:00,2024-05-02T18:03:23+00:00,UTF8,2.030107e+09,0,0,AML_BANK_PREP.sashdat,CASUSER(Chris.Parrish@sas.com),0,Chris.Parrish@sas.com,,2024-04-16T21:17:04+00:00,2.028921e+09,Not Specified


In [4]:
########################
### Create Dataframe ###
########################

dm_inputdf =  conn.CASTable(in_mem_tbl, caslib=caslib)

### print columns for review of model parameters
conn.table.columnInfo(table={"caslib":caslib, "name":in_mem_tbl})

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,account_id,unique identifier,1,double,8,12,,0,0
1,num_transactions,number of transactions financial services cust...,2,double,8,12,,0,0
2,credit_score,customer credit score,3,double,8,12,,0,0
3,marital_status_single,,4,double,8,12,,0,0
4,marital_status_married,,5,double,8,12,,0,0
5,marital_status_divorced,,6,double,8,12,,0,0
6,analytic_partition,,7,double,8,12,,0,0
7,ml_indicator,"indicator for money laundering event no (0), y...",8,double,8,12,,0,0
8,checking_only_indicator,indicator for whether customer only has a chec...,9,double,8,12,,0,0
9,prior_ctr_indicator,indicator for whether the customer has a curre...,10,double,8,12,,0,0


In [5]:
### load CAS actions ###
conn.loadactionset('iml')
conn.loadactionset('astore')

NOTE: Added action set 'iml'.
NOTE: Added action set 'astore'.


[actionset]

 'astore'

+ Elapsed: 0.000812s, user: 0.00022s, sys: 0.000555s, mem: 0.238mb

In [6]:
### create names of tables for action set
astore_tbl = 'aml_bank_astore'
cas_score_tbl = 'aml_bank_score_astore'
cas_out_tbl = str(in_mem_tbl+str('_model'))

In [7]:
#############################
# IML Action Set - Training #
#############################

aml_logit = conn.iml(code=
"""
    /* 1. read data and form design matrix */
	*** note that MatrixCreateFromCAS function will keep/drop in the
	order of the table, not in the order of the column names.
	it is recommended to order column names as they occur in the table ***;

	keep_x = 'KEEP=
			    marital_status_single				
				checking_only_indicator
				prior_ctr_indicator
				address_change_2x_indicator
				cross_border_trx_indicator
				in_person_contact_indicator
				linkedin_indicator
				citizenship_country_risk
				distance_to_employer
				distance_to_bank';
	keep_y = 'KEEP= ml_indicator';
	varNames = {'marital_status_single' 'checking_only_indicator' 'prior_ctr_indicator' 'address_change_2x_indicator' 'cross_border_trx_indicator' 'in_person_contact_indicator' 'linkedin_indicator' 'citizenship_country_risk' 'distance_to_employer' 'distance_to_bank'};
	X = MatrixCreateFromCAS('casuser', 'aml_bank_prep', keep_x);
	y = MatrixCreateFromCAS('casuser', 'aml_bank_prep', keep_y);
	X = j(nrow(X), 1, 1) || X;     /* design matrix: add Intercept column */

	/* 2. define loglikelihood function for binary logistic model */

	start BinLogisticLL(b) global(X, y);
   		z = X*b`;                   /* X*b, where b is a column vector */
   		p = Logistic(z);            /* 1 / (1+exp(-z)) */
   		LL = sum( y#log(p) + (1-y)#log(1-p) );   
   		return( LL );
	finish;
 
	/* 3. Make initial guess and find parameters that maximize the loglikelihood */

	b0 = j(1, ncol(X), 0);         /* initial guess */
	opt = {-1};                    /* find maximum of function */
	call nlpsolve(rc, b, "BinLogisticLL", b0) OPT=opt;   /* use Newton's method to find b that maximizes the LL function */
	print b[c=("Intercept"||varNames) L="Parameter Estimates" F=D8.];
""")
print(aml_logit['Print1.b'])

NOTE: Module BINLOGISTICLL defined.
Parameter Estimates

       COL1      COL2      COL3      COL4      COL5      COL6      COL7  \
0 -6.583794  2.432763  1.347599  1.203979  1.720858  1.638192 -1.948029   

       COL8      COL9     COL10    COL11  
0 -2.001043  0.406579  0.059695 -0.00019  


In [8]:
#####################################
# IML Action Set - Scoring Function #
#####################################

score_model = conn.iml(code=
"""
   /* This function scores one observation at a time */
   start scoreFunc(beta, xn);
      P_ml_indicator1 = exp(beta[1] + beta[2]*xn[1] + beta[3]*xn[2] + beta[4]*xn[3] +
							+ beta[5]*xn[4] + beta[6]*xn[5] + beta[7]*xn[6] +
							+ beta[8]*xn[7] + beta[9]*xn[8] + beta[10]*xn[9] +
							+ beta[11]*xn[10]);
      return P_ml_indicator1;
   finish;

   beta={-6.5838, 2.4328, 1.3476, 1.2040, 1.7209, 1.6382, -1.9480, -2.0010, 0.4066, 0.0597, -0.00019};
   varNames = {'marital_status_single' 'checking_only_indicator' 'prior_ctr_indicator' 'address_change_2x_indicator' 'cross_border_trx_indicator' 'in_person_contact_indicator' 'linkedin_indicator' 'citizenship_country_risk' 'distance_to_employer' 'distance_to_bank'};
   copyVars = varNames || {'ml_indicator'};

   rc = Score('scoreFunc',    /* name of the scoring function */
              beta,          /* scoring constants */
              varNames,        /* input variables */
              'P_ml_indicator1',        /* output variables */
              'aml_bank_prep',        /* input CAS table */
              'aml_bank_score',    /* output CAS table */
              1,             /* pass 1 row at a time */
              copyVars);     /* copy vars from input to output */

	*** create astore ***;
	analytics_store = astore('aml_bank_astore', 'scoreFunc', beta, varNames, 'P_ml_indicator1');
""")

NOTE: Module SCOREFUNC defined.
NOTE: 494455 bytes were written to the table "aml_bank_astore" in the caslib "CASUSER(Chris.Parrish@sas.com)".


In [9]:
#################################
# IML Action Set - Scored Table #
#################################

conn.table.columnInfo(table={"caslib":caslib, "name":'aml_bank_score'})

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,P_ml_indicator1,,1,double,8,12,,0,0
1,marital_status_single,,2,double,8,12,,0,0
2,checking_only_indicator,indicator for whether customer only has a chec...,3,double,8,12,,0,0
3,prior_ctr_indicator,indicator for whether the customer has a curre...,4,double,8,12,,0,0
4,address_change_2x_indicator,indicator for whether customer changed address...,5,double,8,12,,0,0
5,cross_border_trx_indicator,indicator for whether customer had a cross bor...,6,double,8,12,,0,0
6,in_person_contact_indicator,indicator for whether customer has had in-pers...,7,double,8,12,,0,0
7,linkedin_indicator,indicator for whether customer has a current L...,8,double,8,12,,0,0
8,citizenship_country_risk,ordinal risk ranking of customer country of ci...,9,double,8,12,,0,0
9,distance_to_employer,distance between customer's residence and empl...,10,double,8,12,,0,0


In [10]:
conn.table.fetch(table={"caslib":caslib, "name":'aml_bank_score'}, format=True, to=5)

,P_ml_indicator1,marital_status_single,checking_only_indicator,prior_ctr_indicator,address_change_2x_indicator,cross_border_trx_indicator,in_person_contact_indicator,linkedin_indicator,citizenship_country_risk,distance_to_employer,distance_to_bank,ml_indicator
0,0.1318651604,0,0,1,1,1,0,0,0,-0.091215699,-0.895717584,0
1,0.0002972836,0,0,0,0,1,1,1,2,-0.660522669,-0.048730881,0
2,0.0059285697,1,1,0,0,1,1,1,0,-0.233542441,-0.895717584,0
3,0.0575271436,1,0,1,0,1,1,0,1,-0.091215699,-0.774719483,0
4,0.0083942444,1,0,0,1,1,1,1,0,-0.660522669,-0.653721383,0


In [11]:
##########################################
# IML Action Set - Score Table w. Astore #
##########################################

conn.score(table=in_mem_tbl, out=cas_score_tbl, rstore=astore_tbl, copyVars='ALL')
conn.table.fetch(table={"caslib":caslib, "name":cas_score_tbl}, format=True, to=5)

,P_ml_indicator1,account_id,num_transactions,credit_score,marital_status_single,marital_status_married,marital_status_divorced,analytic_partition,ml_indicator,checking_only_indicator,direct_deposit_indicator,citizenship_country_risk,occupation_risk,num_acctbal_chgs_gt2000,distance_to_employer,distance_to_bank,income,primary_transfer_cash,primary_transfer_check,primary_transfer_wire
0,0.1318651604,10729,-0.596972665,0.3220850945,0,0,1,1,0,0,1,0,0,-0.863773416,-0.091215699,-0.895717584,-0.810491149,0,1,0
1,0.0002972836,10730,-0.186827913,1.1999051839,0,1,0,1,0,0,1,2,0,-0.798471061,-0.660522669,-0.048730881,-0.056456725,1,0,0
2,0.0059285697,10731,0.1787358876,-1.756962486,1,0,0,1,0,1,1,0,2,-0.488284872,-0.233542441,-0.895717584,-0.033369697,0,0,1
3,0.0575271436,10732,-0.900123134,-0.879142396,1,0,0,0,0,0,1,1,3,2.3033908305,-0.091215699,-0.774719483,-1.27901674,0,1,0
4,0.0083942444,10733,1.3289244315,0.1988822749,1,0,0,1,0,0,0,0,0,1.3565066742,-0.660522669,-0.653721383,-0.041475909,0,0,1


In [12]:
### print model & results
conn.table.tableInfo(caslib=caslib, wildIgnore=False, name=astore_tbl)
print(conn.astore.describe(rstore={"name":astore_tbl, "caslib":caslib}, epcode=True).Description)
print(conn.astore.describe(rstore={"name":astore_tbl, "caslib":caslib}, epcode=True).InputVariables)
print(conn.astore.describe(rstore={"name":astore_tbl, "caslib":caslib}, epcode=True).OutputVariables)
print(conn.astore.describe(rstore={"name":astore_tbl, "caslib":caslib}, epcode=True).epcode)
model_astore = conn.CASTable(astore_tbl, caslib=caslib)

Basic Information

         Attribute               Value
0  Analytic Engine                 iml
1     Time Created  02May2024:18:52:36
Input Variables

                          Name  Length   Role      Type RawType FormatName
0        marital_status_single     8.0  Input  Interval     Num           
1      checking_only_indicator     8.0  Input  Interval     Num           
2          prior_ctr_indicator     8.0  Input  Interval     Num           
3  address_change_2x_indicator     8.0  Input  Interval     Num           
4   cross_border_trx_indicator     8.0  Input  Interval     Num           
5  in_person_contact_indicator     8.0  Input  Interval     Num           
6           linkedin_indicator     8.0  Input  Interval     Num           
7     citizenship_country_risk     8.0  Input  Interval     Num           
8         distance_to_employer     8.0  Input  Interval     Num           
9             distance_to_bank     8.0  Input  Interval     Num           
Output Variables

    

In [17]:
conn.table.promote(astore_tbl, caslib=caslib)

NOTE: Cloud Analytic Services promoted table AML_BANK_ASTORE in caslib CASUSER(Chris.Parrish@sas.com) to table aml_bank_astore in caslib CASUSER(Chris.Parrish@sas.com).


+ Elapsed: 0.00192s, user: 0.00168s, mem: 0.31mb

In [18]:
store = conn.astore.download(rstore=astore_tbl)
with open('downloaded_astore.sasast','wb') as file:
   file.write(store['blob'])

NOTE: 494455 bytes were downloaded from the table "AML_BANK_ASTORE" in the caslib "CASUSER(Chris.Parrish@sas.com)".


In [13]:
#######################################
### Register Model in Model Manager ###
## Ensure Model Does Not Exist in MM ##
##### Using PZMM Zips Up Metadata #####
#######################################

from sasctl import Session
from sasctl.tasks import register_model, publish_model

In [14]:
### model manager information
model_name = 'logit_action_iml_python'
project_name = 'Anti-Money Laundering'

In [20]:
### create session in cas
sess = Session(hostname=session, token=access_token, client_secret='access_token')
sess

In [ ]:
with sess:
    sas_model = register_model(model_astore, model_name, project_name, force=True, version='latest')